<a href="https://colab.research.google.com/github/hyesukim1/Prediction-of-wine-quality/blob/main/prediction_of_quality_of_wine_white.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 케글 api로 연결하여 데이터 로드

In [ ]:
!pip install kaggle
from google.colab import files
files.upload()

In [ ]:
# ls -1ha kaggle.json

# json file 이동시키기
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# permission warning 방지
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d uciml/red-wine-quality-cortez-et-al-2009

In [ ]:
!ls

----

## 가설


Model
1. Random Forest Classifier
2. Stochastic Gradient Descent Classifier
3. Support Vector Classifier

Model Performance
1. Grid Search CV
2. Cross Validation Score

### 변수 설명

#### 와인의 각종 산은 침샘을 자극해서 식욕을 돋워주고 단맛과 조화를 이뤄 질리지 않는 맛을 만들어주기 때문에 산도가 중요한 변수이다.

1. fixed acidity(고정 산도): 와인에 포함된 대부분의 산이며 와인의 산도를 제어

2. volatile actidity(휘발성 산도): 와인에서 식초와 같은 향을 주는 휘발성 산

3. citric acid(구연산): 와인용 포도에 매우 적은 양을 포함함, 포도의 산미가 약하면 산미 보조제를 더 첨가해 생산함

4. residual sugar(잔류 설탕/grams per liter): 알코올 발효가 끝난 후 와인에 남은 천연 포도당

5. chlorides(염화물): 와인에 포함된 소금의 양으로 짠맛을 내고 신맛을 좌우함

6. free sulfur dioxide(자유 이산화황), total sulfur dioxide(총 이산화황), sulphates(황화합물):특정 박테리아와 효모를 죽여서 와인을 오래 보관하는 역할

7. density(밀도): 바디의 높고 낮음을 표현하는 와인의 무게감을 의미

8. pH(산성도): 와인의 신맛의 정도를 나타냄

9. alcohol(알코올): 와인에 단맛을 주며 바디감에 영향을 줌

10. quality(품질): 예측하려고 하는 변수


# Importing Libraries

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
%matplotlib inline

In [ ]:
# data load
wine_data = pd.read_csv('/content/red-wine-quality-cortez-et-al-2009.zip')

In [ ]:
wine_data.head()

In [ ]:
wine_data.info()
wine_data.shape
wine_data.describe()
wine_data.quality.unique()
wine_data.quality.value_counts()

In [ ]:
# 상관관게 분석
corr_wine = wine_data.corr()
corr_wine = corr_wine.apply(lambda x: round(x, 2))
corr_wine

# 핏쳐가 많기 때문에 히트맵으로 시각화
fig, ax = plt.subplots(figsize=(12, 6))

sns.heatmap(corr_wine,
            annot = True,
            ax = ax,
            linewidths = 0.4,
            linecolor = 'white',
            fmt = '.1f',
            cmap = 'viridis')

In [ ]:
# 밸런스한 데이터 인지 확인
sns.countplot(x = 'quality', data=wine_data)

In [ ]:
sns.boxplot('quality', 'fixed acidity', data = wine_data)

In [ ]:
sns.boxplot('quality', 'volatile acidity', data = wine_data)

In [ ]:
sns.boxplot('quality', 'citric acid', data = wine_data)

In [ ]:
sns.boxplot('quality', 'residual sugar', data = wine_data)

In [ ]:
sns.boxplot('quality', 'chlorides', data = wine_data)

In [ ]:
sns.boxplot('quality', 'free sulfur dioxide', data = wine_data)

In [ ]:
sns.boxplot('quality', 'total sulfur dioxide', data = wine_data)

In [ ]:
sns.boxplot('quality', 'density', data = wine_data)

In [ ]:
sns.boxplot('quality', 'pH', data = wine_data)

In [ ]:
sns.boxplot('quality', 'sulphates', data = wine_data)

In [ ]:
sns.boxplot('quality', 'alcohol', data = wine_data)

In [ ]:
'''
review values 나누기
1, 2, 3 -> Bad == 1
4, 5, 6, 7 -> Average == 2
8, 9, 10 -> Excellent == 3
'''

reviews = []

for i in wine_data['quality']:
  if i >= 3 and i <= 4:
    reviews.append('1')
  elif i >= 5 and i <= 6:
    reviews.append('2')
  elif i >= 7 and i <= 8:
    reviews.append('3')

wine_data['reviews'] = reviews

wine_data

In [ ]:
sns.countplot(x = 'reviews', data=wine_data)

In [ ]:
import collections
collections.Counter(wine_data['reviews'])

In [ ]:
x = wine_data.iloc[:, :11]
y = wine_data['reviews']

In [ ]:
'''
StandardSacler
- 변수들이 너무 스케일이 다르기 때문에 데이터 스케일 조정 필요
- 평균 0, 분산 1로 조정
'''

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc_x = sc.fit_transform(x)
print(sc_x)

In [ ]:
'''
PCA
- 차원의 저주: 머신러닝 훈련 샘플이 각각 수백만개의 특성을 가지고 있을때
  유의미한 특성을 찾기가 어렵고 훈련을 느리게 하기 때문에 결과적으로 성능저하를 야기함
- PCA를 통해서 분산을 최대로 보존할 수 있는 축을 선택해 정보를 가장 적게 손실함
즉, 분산이 최대인 축을 찾고, 이 첫번째 축에 직교하고 남은 분산을 최대한 보존하는 2번째 축을 찾는다
'''

from sklearn.decomposition import PCA
import numpy as np
pca = PCA()
pca_x = pca.fit_transform(x)

plt.figure(figsize=(10, 10))
plt.plot(np.cumsum(pca.explained_variance_ratio_), 'ro-')
plt.grid()

In [ ]:
pca_new = PCA(n_components = 8)
new_x = pca_new.fit_transform(x)
print(new_x)

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(new_x, y, test_size=0.25)

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

# Machine Learning Algorithms

1. Logistic Regression
2. Decision Trees
3. Naive Bayes
4. Random Forests
5. SVM

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score

lr = LogisticRegression()
lr.fit(x_train, y_train)
lr_predict = lr.predict(x_test)

In [ ]:
lr_conf_matrix = confusion_matrix(y_test, lr_predict)
lr_acc_score = accuracy_score(y_test, lr_predict)
print(lr_conf_matrix)
print(lr_acc_score*100)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
dt.fit(x_train, y_train)
dt_pred = dt.predict(x_test)

In [ ]:
dt_conf_matrix = confusion_matrix(y_test, dt_pred)
dt_acc_score = accuracy_score(y_test, dt_pred)
print(dt_conf_matrix)
print(dt_acc_score*100)

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
nb.fit(x_train, y_train)
nb_pred = nb.predict(x_test)

In [ ]:
nb_conf_matrix = confusion_matrix(y_test, nb_pred)
nb_acc_score = accuracy_score(y_test, nb_pred)
print(nb_conf_matrix)
print(nb_acc_score*100)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(x_train, y_train)
rf_predict = rf.predict(x_test)

In [ ]:
rf_conf_matrix = confusion_matrix(y_test, rf_predict)
rf_acc_score = accuracy_score(y_test, rf_predict)
print(rf_conf_matrix)
print(rf_acc_score*100)

In [ ]:
from sklearn.svm import SVC

lin_svc = SVC()
lin_svc.fit(x_train, y_train)
lin_svc = rf.predict(x_test)

In [ ]:
lin_svc_conf_matrix = confusion_matrix(y_test, rf_predict)
lin_svc_acc_score = accuracy_score(y_test, rf_predict)
print(lin_svc_conf_matrix)
print(lin_svc_acc_score*100)